In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import ClassicalRegister, QuantumRegister
from qiskit.quantum_info import Statevector
from qiskit.visualization import array_to_latex
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session, Options
from qiskit import transpile
from qiskit.visualization import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use('bmh')

from qiskit.primitives import SamplerResult

In [ ]:
QiskitRuntimeService.save_account(channel="ibm_quantum", token="8517bec216407d31487d5cb2ce893b43f356a831180bb0807a94effc8f4244187a435a2204a1a78574ac63fb7bea585e87db1165468f943218e9ddbcab3143b3", overwrite=True)

In [ ]:
# A 8x8 binary image represented as a numpy array
image = np.array([[0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 1, 1, 1, 1, 1, 0, 0],
                  [0, 1, 1, 1, 1, 1, 1, 0],
                  [0, 1, 1, 1, 1, 1, 1, 0],
                  [0, 1, 1, 1, 1, 1, 1, 0],
                  [0, 0, 0, 1, 1, 1, 1, 0],
                  [0, 0, 0, 1, 1, 1, 1, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0]])

# Function for plotting the image using matplotlib
def plot_image(img, title: str):
    plt.title(title)
    plt.xticks(range(img.shape[0]))
    plt.yticks(range(img.shape[1]))
    plt.imshow(img, extent=[0, img.shape[0], img.shape[1], 0], cmap='viridis')
    plt.show()

plot_image(image, 'Original Image')

In [ ]:
# Convert the raw pixel values to probability amplitudes
def amplitude_encode(img_data):

    # Calculate the RMS value
    rms = np.sqrt(np.sum(np.sum(img_data**2, axis=1)))

    # Create normalized image
    image_norm = []
    for arr in img_data:
        for ele in arr:
            image_norm.append(ele / rms)

    # Return the normalized image as a numpy array
    return np.array(image_norm)

# Get the amplitude ancoded pixel values
# Horizontal: Original image
image_norm_h = amplitude_encode(image)

# Vertical: Transpose of Original image
image_norm_v = amplitude_encode(image.T)
# Initialize some global variable for number of qubits
data_qb = 6
anc_qb = 1
total_qb = data_qb + anc_qb

# Initialize the amplitude permutation unitary
D2n_1 = np.roll(np.identity(2**total_qb), 1, axis=1)

# Create the circuit for horizontal scan
qc_h = QuantumCircuit(total_qb)
qc_h.initialize(image_norm_h, range(1, total_qb))
qc_h.h(0)
qc_h.unitary(D2n_1, range(total_qb))
qc_h.h(0)
display(qc_h.draw('mpl', fold=-1))

# Create the circuit for vertical scan
qc_v = QuantumCircuit(total_qb)
qc_v.initialize(image_norm_v, range(1, total_qb))
qc_v.h(0)
qc_v.unitary(D2n_1, range(total_qb))
qc_v.h(0)
display(qc_v.draw('mpl', fold=-1))

# Combine both circuits into a single list
circ_list = [qc_h, qc_v]

In [ ]:
# Simulating the cirucits
sv_h = Statevector.from_instruction(qc_h)
sv_v = Statevector.from_instruction(qc_v)


print('Horizontal scan statevector:')
display(array_to_latex(sv_h.data[:30], max_size=30))
print()

print('Vertical scan statevector:')
display(array_to_latex(sv_v.data[:30], max_size=30))

In [ ]:
# Classical postprocessing for plotting the output

# Defining a lambda function for
# thresholding to binary values
threshold = lambda amp: (amp > 1e-15 or amp < -1e-15)

# Selecting odd states from the raw statevector and
# reshaping column vector of size 64 to an 8x8 matrix
edge_scan_h = np.abs(np.array([1 if threshold(sv_h[2*i+1].real) else 0 for i in range(2**data_qb)])).reshape(8, 8)
edge_scan_v = np.abs(np.array([1 if threshold(sv_v[2*i+1].real) else 0 for i in range(2**data_qb)])).reshape(8, 8).T

# Plotting the Horizontal and vertical scans
plot_image(edge_scan_h, 'Horizontal scan output')
plot_image(edge_scan_v, 'Vertical scan output')

In [ ]:
# Combining the horizontal and vertical component of the result
edge_scan_sim = edge_scan_h | edge_scan_v

# Plotting the original and edge-detected images
plot_image(image, 'Original image')
plot_image(edge_scan_sim, 'Edge Detected image')

In [ ]:
# Create a 2x2 image to be run on the hardware
# The pixels in `image_small` correspond to the pixels at
# (6, 2), (6, 3), (7, 2), (7, 3) respectively
image_small = image[6:8, 2:4]

# Plotting the image_small using matplotlib
plot_image(image_small, 'Cropped image')


In [ ]:
# Initialize the number of qubits
data_qb = 2
anc_qb = 1
total_qb = data_qb + anc_qb

# Create the circuit for horizontal scan
qc_small_h = QuantumCircuit(total_qb)
qc_small_h.x(1)
qc_small_h.h(0)

# Decrement gate - START
qc_small_h.x(0)
qc_small_h.cx(0, 1)
qc_small_h.ccx(0, 1, 2)
# Decrement gate - END

qc_small_h.h(0)
qc_small_h.measure_all()
display(qc_small_h.draw('mpl'))

# Create the circuit for vertical scan
qc_small_v = QuantumCircuit(total_qb)
qc_small_v.x(2)
qc_small_v.h(0)

# Decrement gate - START
qc_small_v.x(0)
qc_small_v.cx(0, 1)
qc_small_v.ccx(0, 1, 2)
# Decrement gate - END

qc_small_v.h(0)
qc_small_v.measure_all()
display(qc_small_v.draw('mpl'))

# Combine both circuits into a single list
circ_list = [qc_small_h, qc_small_v]

In [ ]:
service = QiskitRuntimeService()
backend = service.backend("ibm_sherbrooke")
# Transpile the circuits for optimized execution on the backend
qc_small_h_t = transpile(qc_small_h,backend=backend ,optimization_level=3)
qc_small_v_t = transpile(qc_small_v,backend=backend, optimization_level=3)

# Store in a list if needed
circ_list_t = [qc_small_h_t, qc_small_v_t]

# Draw transpiled circuits using modern API
display(circuit_drawer(circ_list_t[0], output="mpl", fold=-1))
display(circuit_drawer(circ_list_t[1], output="mpl", fold=-1))

In [ ]:
with Session(backend=backend) as session:
    sampler = Sampler()
    job = sampler.run(circ_list_t)
    result = job.result()

In [ ]:
quasi_dist = result[0].data.meas

In [ ]:
print(result)

In [ ]:
# Accessing individual results for each circuit
result_circuit_0 = result[0]
result_circuit_1 = result[1]

# Extract quasi-distribution and bitarray
quasi_dist0 = result_circuit_0.data.meas
quasi_dist1 = result_circuit_1.data.meas

# Get total shots from quasi distribution of one of the results
total_shots = quasi_dist0.num_shots


# Function to get counts from quasi distribution
def get_counts_from_quasi_dist(quasi_dist, total_shots):
    counts = {}
    for bitstring, prob in quasi_dist.items():
        count = round(prob * total_shots)
        if count > 0:
            counts[bitstring] = count
    return counts


# Get counts for each circuit using the function
counts0 = get_counts_from_quasi_dist(quasi_dist0, total_shots)
counts1 = get_counts_from_quasi_dist(quasi_dist1, total_shots)

print("Counts for circuit 0:", counts0)
print("Counts for circuit 1:", counts1)

In [ ]:
result[0].data.meas.get_counts().get('0', 0)

In [ ]:
total_shots = 1024  # or the number you assume for scaling
circuit_width = 3   # replace with actual number of qubits

counts = {}
for bitstring, prob in quasi_dist.items():
    count = round(prob * total_shots)
    if count > 0:
        formatted_bitstring = format(int(bitstring, 0), f'0{circuit_width}b')
        counts[formatted_bitstring] = count